In [37]:
# Standard imports. Note: You must pip install nasdaqdatalink 1st
import os
import pandas as pd
import hvplot.pandas
from pathlib import Path

# For API Calls
import nasdaqdatalink
# Do we need requests?
import requests
# For opening zip folder
import shutil 
# For technical analysis
import pandas_ta as ta

In [3]:
# Linking my API key to .env in the same folder. The key is stored in the folder without any quotations around it 

nasdaqdatalink.read_key(filename=".env")

In [4]:
# A function to retrieve a dataframe of counties, zips, etc
def get_regions(regions):
    region_df=nasdaqdatalink.get_table('ZILLOW/REGIONS', region_type=regions)  
    return region_df

# 1. Get the regions data from Zillow REST APIs.   
This contains a list of all counties in the US.

In [5]:
# Using get_regions to retrieve a list of counties
region_df = get_regions('county')
region_df[["county", "state"]] = region_df["region"].str.split(';', 1, expand=True)
region_df["state"] = region_df["state"].str.split(';', 1, expand=True)[0]

#
# Clean up regions data
# Remove ' County' so that we can match the Zillow data with Wikipedia data.
region_df["county"] = region_df["county"].str.replace(" County", "")

# Remove the leading blank space from the 'state' column.
region_df["state"] = region_df['state'].str[1:]

# Clean up region_id datatype.
region_df['region_id']=region_df['region_id'].astype(int)

# Check data for region_df
display(region_df.head())
display(region_df.tail())

,region_id,region_type,region,county,state
None,,,,,
0,999,county,Durham County; NC; Durham-Chapel Hill,Durham,NC
1,998,county,Duplin County; NC,Duplin,NC
2,997,county,Dubois County; IN; Jasper,Dubois,IN
3,995,county,Donley County; TX,Donley,TX
4,993,county,Dimmit County; TX,Dimmit,TX


,region_id,region_type,region,county,state
None,,,,,
2886,1003,county,Elmore County; AL; Montgomery,Elmore,AL
2887,1002,county,Elbert County; GA,Elbert,GA
2888,1001,county,Elbert County; CO; Denver-Aurora-Lakewood,Elbert,CO
2889,1000,county,Echols County; GA; Valdosta,Echols,GA
2890,100,county,Bibb County; AL; Birmingham-Hoover,Bibb,AL


# 2. Get the Zillow sales data.  
In this example, we read in Zillow sales data in the form of a CSV file.  

In [6]:
# Get the Zillow sales data. 
# The actual API call using the SDK.
# Instructions can be found here https://data.nasdaq.com/databases/ZILLOW/usage/quickstart/python
# Replace 'quandl' w/ 'nasdaqdatalink
# Turned into a function to prevent constant re-downloading massive csv

def get_zillow_data():
    data = nasdaqdatalink.export_table('ZILLOW/DATA', indicator_id='ZSFH', region_id=list(region_df['region_id']),filename='db.zip')
    
    # Unzipping database from API call
    shutil.unpack_archive('db.zip')
    return data        

In [7]:
# Reading in Database
zillow_data=pd.read_csv(
    Path('ZILLOW_DATA_d5d2ff90eb7172dbde848ea36de12dfe.csv')
)

# Check the Zillow sales data
display(zillow_data.head())
display(zillow_data.tail())

,indicator_id,region_id,date,value
0,ZSFH,100,2007-11-30,123760.0
1,ZSFH,100,2007-12-31,123754.0
2,ZSFH,100,2008-01-31,123605.0
3,ZSFH,100,2008-02-29,123393.0
4,ZSFH,100,2008-03-31,123095.0


,indicator_id,region_id,date,value
669311,ZSFH,999,2022-02-28,390111.0
669312,ZSFH,999,2022-03-31,401621.0
669313,ZSFH,999,2022-04-30,411421.0
669314,ZSFH,999,2022-05-31,422028.0
669315,ZSFH,999,2022-06-30,430509.0


In [8]:
## Merge the Region dataframe with the Zillow sales data
zillow_merge_df = pd.merge(region_df, zillow_data, on=['region_id'])

# Check the merged Zillow data
zillow_merge_df.head()

,region_id,region_type,region,county,state,indicator_id,date,value
0,999,county,Durham County; NC; Durham-Chapel Hill,Durham,NC,ZSFH,1997-02-28,139430.0
1,999,county,Durham County; NC; Durham-Chapel Hill,Durham,NC,ZSFH,1997-03-31,139459.0
2,999,county,Durham County; NC; Durham-Chapel Hill,Durham,NC,ZSFH,1997-04-30,139659.0
3,999,county,Durham County; NC; Durham-Chapel Hill,Durham,NC,ZSFH,1997-05-31,139887.0
4,999,county,Durham County; NC; Durham-Chapel Hill,Durham,NC,ZSFH,1997-06-30,140303.0


# 3. Get the county coordinates data.
We couldn't find the county coordinates from Zillow, so we sourced the data from Wikipedia.   We are going to have to merge the data with Zillow based on county and state. 

In [9]:
# Read in county data with coordinates
county_coordinates_df=pd.read_csv(
    Path('counties_w_coordinates.csv')
)

# Clean up data.
# We need to rename the columns so that we can merge our Zillow data set 
# with the county coordinates data.   The dataframes will be merged against 'county' and 'state'. 
county_coordinates_df = county_coordinates_df.rename(columns={"County\xa0[2]" : "county"})
# county_coordinates_df = county_coordinates_df.rename(columns={"region" : "region"})
county_coordinates_df = county_coordinates_df.rename(columns={"State" : "state"})

# Remove degrees 
county_coordinates_df["Latitude"] = county_coordinates_df["Latitude"].str.replace("°", "")
county_coordinates_df["Longitude"] = county_coordinates_df["Longitude"].str.replace("°", "")

# Remove + sign for Latitude and Longitude
county_coordinates_df["Latitude"] = county_coordinates_df["Latitude"].str.replace("+", "")
county_coordinates_df["Longitude"] = county_coordinates_df["Longitude"].str.replace("+", "")

# Some of the data uses unicode hyphens which causes problems when trying to convert the Longitude and Latitude to float.
county_coordinates_df["Latitude"] = county_coordinates_df["Latitude"].str.replace('\U00002013', '-')
county_coordinates_df["Longitude"] = county_coordinates_df["Longitude"].str.replace('\U00002013', '-')

# Convert Longitude and Latitude to float so we can display on the map. 
county_coordinates_df["Latitude"] = county_coordinates_df["Latitude"].astype(float)
county_coordinates_df["Longitude"] = county_coordinates_df["Longitude"].astype(float)

# Check the county coordinates data
county_coordinates_df.head()

C:\Users\gtkhh\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
C:\Users\gtkhh\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.


,Sort [1],state,FIPS,county,County Seat(s) [3],Population,Land Area,Land Area.1,Water Area,Water Area.1,Total Area,Total Area.1,Latitude,Longitude
0,NaN,NaN,NaN,NaN,NaN,-2010,km²,mi²,km²,mi²,km²,mi²,NaN,NaN
1,1.0,AL,1001.0,Autauga,Prattville,"54,571","1,539.58",594.436,25.776,9.952,"1,565.36",604.388,32.536382,-86.644490
2,2.0,AL,1003.0,Baldwin,Bay Minette,"182,265","4,117.52","1,589.78","1,133.19",437.527,"5,250.71","2,027.31",30.659218,-87.746067
3,3.0,AL,1005.0,Barbour,Clayton,"27,457","2,291.82",884.876,50.865,19.639,"2,342.68",904.515,31.870670,-85.405456
4,4.0,AL,1007.0,Bibb,Centreville,"22,915","1,612.48",622.582,9.289,3.587,"1,621.77",626.169,33.015893,-87.127148


In [111]:
# Merge the Zillow data and county coordinates data.
master_df = pd.merge(zillow_merge_df, county_coordinates_df, on=['county', 'state'])

master_df['date']=pd.to_datetime(master_df['date'])

# Check the master data
master_df

,region_id,region_type,region,county,state,indicator_id,date,value,Sort [1],FIPS,County Seat(s) [3],Population,Land Area,Land Area.1,Water Area,Water Area.1,Total Area,Total Area.1,Latitude,Longitude
0,999,county,Durham County; NC; Durham-Chapel Hill,Durham,NC,ZSFH,1997-02-28,139430.0,1922.0,37063.0,Durham,"267,587",740.673,285.975,30.798,11.891,771.471,297.866,36.036589,-78.877919
1,999,county,Durham County; NC; Durham-Chapel Hill,Durham,NC,ZSFH,1997-03-31,139459.0,1922.0,37063.0,Durham,"267,587",740.673,285.975,30.798,11.891,771.471,297.866,36.036589,-78.877919
2,999,county,Durham County; NC; Durham-Chapel Hill,Durham,NC,ZSFH,1997-04-30,139659.0,1922.0,37063.0,Durham,"267,587",740.673,285.975,30.798,11.891,771.471,297.866,36.036589,-78.877919
3,999,county,Durham County; NC; Durham-Chapel Hill,Durham,NC,ZSFH,1997-05-31,139887.0,1922.0,37063.0,Durham,"267,587",740.673,285.975,30.798,11.891,771.471,297.866,36.036589,-78.877919
4,999,county,Durham County; NC; Durham-Chapel Hill,Durham,NC,ZSFH,1997-06-30,140303.0,1922.0,37063.0,Durham,"267,587",740.673,285.975,30.798,11.891,771.471,297.866,36.036589,-78.877919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
628666,100,county,Bibb County; AL; Birmingham-Hoover,Bibb,AL,ZSFH,2022-02-28,161462.0,4.0,1007.0,Centreville,"22,915","1,612.48",622.582,9.289,3.587,"1,621.77",626.169,33.015893,-87.127148
628667,100,county,Bibb County; AL; Birmingham-Hoover,Bibb,AL,ZSFH,2022-03-31,162369.0,4.0,1007.0,Centreville,"22,915","1,612.48",622.582,9.289,3.587,"1,621.77",626.169,33.015893,-87.127148
628668,100,county,Bibb County; AL; Birmingham-Hoover,Bibb,AL,ZSFH,2022-04-30,163859.0,4.0,1007.0,Centreville,"22,915","1,612.48",622.582,9.289,3.587,"1,621.77",626.169,33.015893,-87.127148
628669,100,county,Bibb County; AL; Birmingham-Hoover,Bibb,AL,ZSFH,2022-05-31,164684.0,4.0,1007.0,Centreville,"22,915","1,612.48",622.582,9.289,3.587,"1,621.77",626.169,33.015893,-87.127148


# 4. Display in a Map

In [112]:
# Get mean data by state and county
county_df = master_df.groupby(["state", "county"]).mean()

# Divide price by 1000 so that it looks better on map.
county_df["value"] = county_df["value"] / 1000

# Check data
display(county_df.head())
display(county_df.tail())

region_id       value  Sort [1]    FIPS   Latitude  Longitude
state county                                                                
AL    Autauga     1524.0  144.376453       1.0  1001.0  32.536382 -86.644490
      Baldwin     1525.0  172.793462       2.0  1003.0  30.659218 -87.746067
      Barbour     1531.0   88.204607       3.0  1005.0  31.870670 -85.405456
      Bibb         100.0  117.338341       4.0  1007.0  33.015893 -87.127148
      Blount       883.0  126.394141       5.0  1009.0  33.977448 -86.567246

region_id       value  Sort [1]     FIPS   Latitude  \
state county                                                            
WY    Sweetwater     3036.0  217.047598    3139.0  56037.0  41.660339   
      Teton           745.0  850.719510    3140.0  56039.0  44.049321   
      Uinta           760.0  183.118510    3141.0  56041.0  41.284726   
      Washakie       2593.0  162.015983    3142.0  56043.0  43.878831   
      Weston         1486.0  169.969867    3143.0  56045.0  43.846213   

                   Longitude  
state county                  
WY    Sweetwater -108.875676  
      Teton      -110.588102  
      Uinta      -110.558947  
      Washakie   -107.669052  
      Weston     -104.570020

In [113]:
county_df.hvplot.points(
    'Longitude',
    'Latitude',
    geo=True,
    size='value',
    color='value',
    tiles='OSM',
    height=700,
    width=1200)

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (value)

# The MAC/D

In [114]:
filtered_df = master_df[['date','county','state','value']]

filtered_df

,date,county,state,value
0,1997-02-28,Durham,NC,139430.0
1,1997-03-31,Durham,NC,139459.0
2,1997-04-30,Durham,NC,139659.0
3,1997-05-31,Durham,NC,139887.0
4,1997-06-30,Durham,NC,140303.0
...,...,...,...,...
628666,2022-02-28,Bibb,AL,161462.0
628667,2022-03-31,Bibb,AL,162369.0
628668,2022-04-30,Bibb,AL,163859.0
628669,2022-05-31,Bibb,AL,164684.0


In [115]:
exploratory_df=filtered_df.groupby('date').count()

exploratory_df

,county,state,value
date,,,
1996-01-31,927,927,927
1996-02-29,933,933,933
1996-03-31,935,935,935
1996-04-30,935,935,935
1996-05-31,939,939,939
...,...,...,...
2022-02-28,2665,2665,2665
2022-03-31,2662,2662,2662
2022-04-30,2671,2671,2671


In [116]:
# Create new DataFrame with summed county markets to represent the entire nation
nationwide_df = filtered_df.groupby(filtered_df['date']).agg({'value':'sum'})

display(nationwide_df)

# Must divide 'values' by number of counties that make up said value so data isn't skewed by county number
nationwide_df['avg'] = nationwide_df['value']/exploratory_df['county']

display(nationwide_df)

,value
date,
1996-01-31,85189130.0
1996-02-29,85662331.0
1996-03-31,85837853.0
1996-04-30,85991185.0
1996-05-31,86695047.0
...,...
2022-02-28,607642486.0
2022-03-31,614296151.0
2022-04-30,624782299.0


,value,avg
date,,
1996-01-31,85189130.0,91897.659115
1996-02-29,85662331.0,91813.859593
1996-03-31,85837853.0,91805.190374
1996-04-30,85991185.0,91969.181818
1996-05-31,86695047.0,92326.993610
...,...,...
2022-02-28,607642486.0,228008.437523
2022-03-31,614296151.0,230764.895192
2022-04-30,624782299.0,233913.253089


In [133]:
# Uses pandas_ta MACD funtion
nationwide_macd_df = nationwide_df.ta.macd(close='avg', fast=6, slow=12, signal=4, append=True)
# Making DataFrame look nice
nationwide_macd_df = nationwide_macd_df.rename(columns={'MACD_6_12_4':'fast_ema','MACDh_6_12_4':'signal','MACDs_6_12_4':'slow_ema'})
# Divide by 1000 so it looks more like a momentum indicator
nationwide_macd_df = nationwide_macd_df/1000

nationwide_macd_df

,fast_ema,signal,slow_ema
date,,,
1996-01-31,NaN,NaN,NaN
1996-02-29,NaN,NaN,NaN
1996-03-31,NaN,NaN,NaN
1996-04-30,NaN,NaN,NaN
1996-05-31,NaN,NaN,NaN
...,...,...,...
2022-02-28,9.083628,0.568550,8.515078
2022-03-31,9.270278,0.453120,8.817158
2022-04-30,9.390772,0.344168,9.046603


In [136]:
# Graph

nationwide_macd_df.hvplot(title='US Housing Market Momentum', ylabel='Momentum')

:NdOverlay   [Variable]
   :Curve   [date]   (value)